In [11]:
import camelot as camelot
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gop
from geopy.geocoders import Nominatim
import folium as fo
from folium.plugins import MarkerCluster
from folium import plugins
from folium.plugins import HeatMap



In [12]:
# Extract tables 

file = "https://cpcb.nic.in//upload/Downloads/AQI_Bulletin_20220307.pdf"
tables = camelot.read_pdf(file, pages='all', strip_text='\n', flag_size=True)

In [13]:
# Define headers
def filter_df(df):
    df.columns = ['City', 'Air Quality', 'Index Value', 'Prominent Pollutant', 'No. of Monitoring Stations']
    df.reset_index().drop(columns= 'index', axis=1, inplace=True)
    return df

In [14]:
# First table to dataframe
tables[0].df

,0,1,2,3,4,5
0,S.No,City,Air Quality,Index Value,Prominent Pollutant,Based on Numberof MonitoringStations
1,1,Agartala,Poor,225,PM2.5,1
2,2,Agra,Moderate,140,"PM, O, PM1032.5",6
3,3,Ahmedabad,Poor,239,PM2.5,8
4,4,Aizawl,Satisfactory,70,PM10,1
5,5,Ajmer,Satisfactory,91,PM10,1
6,6,Alwar,Satisfactory,83,PM10,1
7,7,Amaravati,Satisfactory,56,PM10,1
8,8,Amritsar,Satisfactory,77,PM10,1
9,9,Ankleshwar,Moderate,196,PM2.5,1


In [15]:
# Even number of dataframes
table_list = []
for num, table in enumerate(tables):
    if  num % 2 == 0:
        table_df = table.df
        table_list.append(table_df.drop(0).drop(0, axis=1))

In [16]:
# Concatenate all dateframes
df = pd.concat(table_list)

In [17]:
# Add headers
df.pipe(filter_df)

,City,Air Quality,Index Value,Prominent Pollutant,No. of Monitoring Stations
1,Agartala,Poor,225,PM2.5,1
2,Agra,Moderate,140,"PM, O, PM1032.5",6
3,Ahmedabad,Poor,239,PM2.5,8
4,Aizawl,Satisfactory,70,PM10,1
5,Ajmer,Satisfactory,91,PM10,1
...,...,...,...,...,...
7,Vatva,Poor,203,PM2.5,1
8,Vijayapura,Satisfactory,51,PM10,1
9,Visakhapatnam,Moderate,130,PM10,1
10,Yadgir,Satisfactory,53,PM10,1


In [18]:
# Add date 

df['Date'] = pd.to_datetime('today').strftime("%d/%m/%Y")
df

,City,Air Quality,Index Value,Prominent Pollutant,No. of Monitoring Stations,Date
1,Agartala,Poor,225,PM2.5,1,28/03/2022
2,Agra,Moderate,140,"PM, O, PM1032.5",6,28/03/2022
3,Ahmedabad,Poor,239,PM2.5,8,28/03/2022
4,Aizawl,Satisfactory,70,PM10,1,28/03/2022
5,Ajmer,Satisfactory,91,PM10,1,28/03/2022
...,...,...,...,...,...,...
7,Vatva,Poor,203,PM2.5,1,28/03/2022
8,Vijayapura,Satisfactory,51,PM10,1,28/03/2022
9,Visakhapatnam,Moderate,130,PM10,1,28/03/2022
10,Yadgir,Satisfactory,53,PM10,1,28/03/2022


In [43]:
# load previous data

df_base = pd.read_csv("AQI.csv")

df_base.head()


# Append latest data

frames = [df, df_base]
df1 = pd.concat(frames)




# Export to csv
df1.to_csv("AQI.csv",index = False)
df1



,City,Air Quality,Index Value,Prominent Pollutant,No. of Monitoring Stations,Date
1,Agartala,Poor,225,PM2.5,1,28/03/2022
2,Agra,Moderate,140,"PM, O, PM1032.5",6,28/03/2022
3,Ahmedabad,Poor,239,PM2.5,8,28/03/2022
4,Aizawl,Satisfactory,70,PM10,1,28/03/2022
5,Ajmer,Satisfactory,91,PM10,1,28/03/2022
...,...,...,...,...,...,...
281,Vatva,Poor,203,PM2.5,1,28/03/2022
282,Vijayapura,Satisfactory,51,PM10,1,28/03/2022
283,Visakhapatnam,Moderate,130,PM10,1,28/03/2022
284,Yadgir,Satisfactory,53,PM10,1,28/03/2022


In [44]:
#create 2 new columns to store lat/long - initalise to null
df1['location_lat'] = ""
df1['location_long'] = ""

#print first rows to sample
df1.head()

,City,Air Quality,Index Value,Prominent Pollutant,No. of Monitoring Stations,Date,location_lat,location_long
1,Agartala,Poor,225,PM2.5,1,28/03/2022,,
2,Agra,Moderate,140,"PM, O, PM1032.5",6,28/03/2022,,
3,Ahmedabad,Poor,239,PM2.5,8,28/03/2022,,
4,Aizawl,Satisfactory,70,PM10,1,28/03/2022,,
5,Ajmer,Satisfactory,91,PM10,1,28/03/2022,,


In [21]:


'''
**Get Lat/Long Data with GeoPy**
---------------------

the code below calls a geopy API using a concatenated column of address values. We use this column as a query key 
to pull back cooresponding lat/long coordinates.
'''

geolocator = Nominatim(user_agent="myApp")

for i in df1.index:
    try:
        #tries fetch address from geopy
        location = geolocator.geocode(df1['City'][i])
        
        #append lat/long to column using dataframe location
        df1.loc[i,'location_lat'] = location.latitude
        df1.loc[i,'location_long'] = location.longitude
    
    except:
        #catches exception for the case where no value is returned
        #appends null value to column
        df1.loc[i,'location_lat'] = ""
        df1.loc[i,'location_long'] = ""
        

#print first rows as sample
df1.head()



,City,Air Quality,Index Value,Prominent Pollutant,No. of Monitoring Stations,Date,location_lat,location_long
1,Agartala,Poor,225,PM2.5,1,28/03/2022,,
2,Agra,Moderate,140,"PM, O, PM1032.5",6,28/03/2022,,
3,Ahmedabad,Poor,239,PM2.5,8,28/03/2022,,
4,Aizawl,Satisfactory,70,PM10,1,28/03/2022,,
5,Ajmer,Satisfactory,91,PM10,1,28/03/2022,,


In [22]:
#remove duplicate addresses (new concat column) but keep first instance
df1['location_lat'].replace('', np.nan, inplace=True)

df1.dropna(subset=['location_lat'], inplace=True)

df1

,City,Air Quality,Index Value,Prominent Pollutant,No. of Monitoring Stations,Date,location_lat,location_long
0,Agartala,Poor,225,PM2.5,1,28/03/2022,23.831238,91.282382
13,Bagalkot,Good,50,PM2.5,1,28/03/2022,16.185317,75.696792
14,Ballabgarh,Moderate,101,PM10,1,28/03/2022,28.291714,77.360407
15,Bathinda,Moderate,108,PM10,1,28/03/2022,30.179115,75.047102
16,Bengaluru,Satisfactory,69,PM10,8,28/03/2022,12.976794,77.590082
...,...,...,...,...,...,...,...,...
138,Vatva,Poor,203,PM2.5,1,28/03/2022,22.958998,72.629798
139,Vijayapura,Satisfactory,51,PM10,1,28/03/2022,16.666667,75.916667
140,Visakhapatnam,Moderate,130,PM10,1,28/03/2022,17.723128,83.301284
141,Yadgir,Satisfactory,53,PM10,1,28/03/2022,16.768904,77.138038


In [33]:
# Check percentage of rows geocoded
print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(df1["location_lat"])) / len(df1)) * 100))



100.0% of addresses were geocoded!


In [42]:
# Field with NaN
df1[df1.isna().any(axis=1)]

,City,Air Quality,Index Value,Prominent Pollutant,No. of Monitoring Stations,Date,location_lat,location_long


In [47]:
 # Remove rows with wrongly coded.


df2 = pd.read_csv("AQI_.csv")

df2.drop(df2[df2['location_long'] <= 30].index, inplace = True)
df2

 #min(df1.location_long)

,City,Air Quality,Index Value,Prominent Pollutant,No. of Monitoring Stations,Date,location_lat,location_long
0,Agartala,Poor,225,PM2.5,1,28/03/2022,23.831238,91.282382
1,Bagalkot,Good,50,PM2.5,1,28/03/2022,16.185317,75.696792
2,Ballabgarh,Moderate,101,PM10,1,28/03/2022,28.291714,77.360407
3,Bathinda,Moderate,108,PM10,1,28/03/2022,30.179115,75.047102
4,Bengaluru,Satisfactory,69,PM10,8,28/03/2022,12.976794,77.590082
...,...,...,...,...,...,...,...,...
120,Vatva,Poor,203,PM2.5,1,28/03/2022,22.958998,72.629798
121,Vijayapura,Satisfactory,51,PM10,1,28/03/2022,16.666667,75.916667
122,Visakhapatnam,Moderate,130,PM10,1,28/03/2022,17.723128,83.301284
123,Yadgir,Satisfactory,53,PM10,1,28/03/2022,16.768904,77.138038


In [48]:
# Export to csv
df2.to_csv("AQI_.csv",index = False)

In [56]:
# Create a map
m = fo.Map(location=[20, 80], tiles='openstreetmap', zoom_start=2)

# add marker one by one on the map
for i in range(0,len(df2)):
   fo.Marker(
     location=[df2.iloc[i]['location_lat'], df2.iloc[i]['location_long']],
      popup=[df2.iloc[i]['City'],df2.iloc[i]['Date'],df2.iloc[i]['Air Quality']],
   ).add_to(m)




locations = list(zip(df2["location_lat"], df2["location_long"]))



# Create a folium marker cluster


cluster = MarkerCluster(locations=locations)

# Add marker cluster to map
cluster.add_to(m)


# Display the map
m

In [57]:
# Add heatmap

HeatMap(locations).add_to(m)

m